In [1]:
# Imports
import argparse
import os
import sys
import time
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from _datetime import datetime
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib import rcParams
from PIL import Image
from sklearn.utils import resample
from scipy.stats import pearsonr
import scipy.stats as stats
import scipy

In [3]:
# Local imports
sys.path.append("/home/users/benhutch/skill-maps")
import dictionaries as dicts

sys.path.append("/home/users/benhutch/skill-maps/python")
import functions as fnc

In [4]:
# Imports of historical functions
sys.path.append("/home/users/benhutch/skill-maps-differences")
import functions as hist_fnc

In [5]:
# Set up the parameters
# Test run with psl dcppA-hindcast raw data and historical data
variable = "psl"
obs_var_name = "msl"
region = "global"
region_grid = dicts.gridspec_global
forecast_range = "2-9"
season = "DJFM"
model_season = "DJFM"
start_year = "1960"
end_year = "2022"

# Set up the paths
observations_path = dicts.obs
dcpp_models = dicts.models
historical_models = dicts.psl_models
base_dir = dicts.base_dir
base_dir_historical = dicts.base_dir_historical
output_dir = dicts.plots_dir
save_dir = dicts.save_dir

AttributeError: module 'dictionaries' has no attribute 'base_dir_historical'

In [ ]:
# Process the observations
obs = fnc.process_observations(variable, region, region_grid, forecast_range, season,
                                observations_path, obs_var_name, plev=None)

In [ ]:
# Load the historical data
hist_data = hist_fnc.load_processed_historical_data(base_dir_historical, historical_models,
                                                    variable, region, forecast_range, season)

# Extract the data for the variable from the historical data
hist_data, _ = hist_fnc.extract_historical_data(hist_data, variable)

In [ ]:
# Load the DCPP data
dcpp_data = fnc.load_data(base_dir, dcpp_models, variable, region, 
                            forecast_range, season, level=None)

# Extract the data for the variable from the DCPP data
dcpp_data, _ = fnc.process_data(dcpp_data, variable)

In [ ]:
# Now we want to make sure that the time period 
# is the same for all of the data
# and extract the data into arrays
